# Image and Label Overlay Viewer

This notebook allows you to:
1. Visualize image and label pairs from specified directories.
2. Interactively scroll through the dataset.
3. Batch save overlay images to a specified output directory.

In [1]:
%matplotlib inline
import os
from pathlib import Path
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Layout
from skimage.color import label2rgb

# Set default figure params
plt.rcParams["figure.figsize"] = [10, 10]

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
# --- CONFIGURATION ---
# Paths to your data. Modify these to point to your specific folders.
# Can be absolute paths or relative to this notebook.

# Example paths (modify these):
base_data_dir = Path("../../dat/augmented_training_data")
images_dir = base_data_dir / "images"
labels_dir = base_data_dir / "labels"

# Output directory for saved overlays
output_dir = Path("overlays_output")

print(f"Images Directory: {images_dir.resolve()}")
print(f"Labels Directory: {labels_dir.resolve()}")

In [ ]:
# --- LOAD DATA ---

# Supported extensions
extensions = ["*.tif", "*.tiff", "*.png", "*.jpg"]

image_files = []
if images_dir.exists():
    for ext in extensions:
        image_files.extend(list(images_dir.glob(ext)))
    image_files = sorted(image_files)
else:
    print(f"Error: {images_dir} does not exist.")

# Match labels to images by filename
pairs = []
for img_path in image_files:
    # Assume label has the same filename
    label_path = labels_dir / img_path.name
    if label_path.exists():
        pairs.append((img_path, label_path))
    else:
        # Try checking if maybe extension is different?
        # For now, stick to strict filename matching
        pass

print(f"Found {len(pairs)} matched image-label pairs.")
if len(pairs) == 0:
    print("No pairs found! Please check your paths.")

In [ ]:
# --- INTERACTIVE VIEWER ---


def load_and_prep(img_path, label_path):
    image = tifffile.imread(img_path)
    label = tifffile.imread(label_path)

    # Handle dimensions (take first slice if 3D)
    if image.ndim > 2:
        image = image[0]
    if label.ndim > 2:
        label = label[0]

    # Normalize image for visualization (percentile)
    p_low, p_high = np.percentile(image, (1, 99))
    scaling = p_high - p_low
    if scaling == 0:
        scaling = 1
    img_norm = np.clip((image - p_low) / scaling, 0, 1)

    return img_norm, label


def view_overlay(index):
    if not pairs:
        print("No data to show.")
        return

    img_path, label_path = pairs[index]

    try:
        image, label = load_and_prep(img_path, label_path)

        # Create overlay
        # label2rgb is great for this: puts labels in color on top of grayscale image
        overlay = label2rgb(label, image=image, bg_label=0, alpha=0.3)

        plt.figure(figsize=(10, 10))
        plt.imshow(overlay)
        plt.title(f"Index: {index} | {img_path.name}")
        plt.axis("off")
        plt.show()
    except Exception as e:
        print(f"Error loading {img_path.name}: {e}")


if pairs:
    interact(
        view_overlay,
        index=IntSlider(
            min=0,
            max=len(pairs) - 1,
            step=1,
            value=0,
            description="Image Idx:",
            layout=Layout(width="80%"),
        ),
    )
else:
    print("No pairs available for interactive view.")

In [ ]:
# --- BATCH SAVE ---


def batch_save_overlays(out_dir):
    out_path = Path(out_dir)
    out_path.mkdir(exist_ok=True, parents=True)
    print(f"Saving {len(pairs)} overlays to {out_path.resolve()}...")

    for i, (img_path, label_path) in enumerate(pairs):
        try:
            image, label = load_and_prep(img_path, label_path)
            overlay = label2rgb(label, image=image, bg_label=0, alpha=0.3)

            # Save
            save_name = out_path / f"overlay_{img_path.stem}.png"
            plt.imsave(save_name, overlay)

            if (i + 1) % 10 == 0:
                print(f"Saved {i + 1}/{len(pairs)}")
        except Exception as e:
            print(f"Failed to save {img_path.name}: {e}")

    print("Finished saving.")


# Run this manually when you want to save all
# batch_save_overlays(output_dir)